ADAIX - 20-22 May 2018, Valencia - Spain

### Session II - Machine Learning in Practice: 
# Regression - II: Photometric redshift estimation 
by *Emille E. O. Ishida* and *Alexandre Boucaud*


In this example we are going to estimate photometric redshfits based on measurements of r-band magnitude and colours. 

## The Data

We shall work with the `Teddy` catalog presented in [Beck *et al.*, 2017](https://arxiv.org/abs/1701.08748).   
`Teddy` was build from SDSS-DR12 data release.  The complete catalog is composed of 4 samples with increasing levels of non-representativeness.  You can take a look in all the available samples in the [COINtoolbox](https://github.com/COINtoolbox/photoz_catalogues).

We begin by analysing `TeddyA`, a well behaved, high SNR sample.

In [ ]:
# read TeddyA
TeddyA = np.loadtxt('')

------------------------------------------------------------------------------------------------------------------
Summary:

## Machine Learning Model    

**Task**:  Estimate photometric redshift based on measurements of magnitudes and colours.  

       input: r-band magnitude and 4 different colours in SDSS broad-band filters
       output: estimated photometric redshifts=
       
**Task Category**:   Regression

**Data**: `Teddy` catalog from [Beck *et al*., 2017](https://arxiv.org/abs/1701.08748)  
        
        5 Features, x 
            mag_r       magnitude in SDSS r-band
              u-g       u - g colour in SDSS system
              g-r       g - r colour in SDSS system
              r-i       r - i colour in SDSS system
              i-z       i - z colour in SDSS system
              
    
        1 response variable (label), y    
            redshift
            
**Machine Learning category**:  

        Supervised Learning

**Set of possible samples**:  

        74309 galaxies
        
**Set of possible labels**:

        positive real numbers (aditional constraints are observational)
        
**Learner**:
    
        1 - Linear Regression     
        2 - Random Forest
        
**Loss function**:

        Sum of squared residuals (Ordinary Least Square)
         